In [ ]:
import numpy as np
import pandas as pd
import re
import requests
import string
import argparse
from tqdm import tqdm
from bs4 import BeautifulSoup

def parse_interview(url, data):
  page = requests.get(url)
  soup = BeautifulSoup(page.text, 'html.parser')
  section = soup.find(attrs={'style':'padding: 10px;', 'valign':'top'})
  event = soup.find('h1').get_text()
  date = soup.find('h2').get_text()
  items = soup.find_all("h3")
  names = [item.get_text() for item in items]
  for p in soup.find_all(["strong", "i", "h1", "h2", "h3", "br", "a"]):
    p.decompose()
  paragraphs = section.find_all(string=True)
  output = ""
  for p in paragraphs:
    text = p.get_text()
    output = output + text
  data.append([output, event, date, names])

def parse_player(url, data):
  page = requests.get(url)
  soup = BeautifulSoup(page.text, 'html.parser')
  table = soup.find('table', attrs={'width':'100%', 'cellspacing':'0', 
	                  'cellpadding':'3', 'border':'0'})
  if table is None:
    return None
  links = table.find_all('a', href=True)
  for link in links:
    parse_interview(link['href'], data)

def parse_letter(url, data):
  page = requests.get(url)
  soup = BeautifulSoup(page.text, 'html.parser')
  table = soup.find('table', attrs={'width':'100%', 'cellspacing':'0', 
	                  'cellpadding':'3', 'border':'0'})
  if table is None:
    return None
  links = table.find_all('a', href=True)
  for link in tqdm(links):
    parse_player(link['href'], data)

def parse_sport(url):
  page = requests.get(url)
  soup = BeautifulSoup(page.text, 'html.parser')
  table = soup.find_all('table', attrs={'width':'100%', 'cellspacing':'0', 
	                  'cellpadding':'5', 'border':'0'})[0]
  links = table.find_all('a', href=True)
  data = []
  for link in links:
    parse_letter(link['href'], data)
  return data

def scrape():
  
  data = parse_sport("https://www.asapsports.com/showcat.php?id=2")
  df = pd.DataFrame(data, columns=['text', 'event', 'date', 'names'])
  df.to_csv("corpus_creation/interviews_raw.csv", index=False)

100%|██████████| 17/17 [00:29<00:00,  1.74s/it]
